In [1]:
import numpy as np
from scipy.stats import norm

n = 100
x = np.random.rand(n, 3)
y = 1.0 + x[:,0] + 2*x[:,1] - 0*x[:,2] + norm.rvs(size=n)

In [2]:
import statsmodels.api as sm

X = sm.add_constant(x)
model = sm.OLS(y, X).fit()

conf_int = model.conf_int(alpha = 0.05)
for i in range(4):
    print('The 95%% CI for beta%d is:' % i , conf_int[i])

The 95% CI for beta0 is: [0.4840894  2.01568654]
The 95% CI for beta1 is: [-0.26096879  1.48735579]
The 95% CI for beta2 is: [0.9486859  2.57695948]
The 95% CI for beta3 is: [-0.68833389  0.85484193]


In [3]:
SSE = np.sum(model.resid ** 2)
SSR = np.sum((model.fittedvalues - np.mean(y)) ** 2)
n = len(y)
p = 3  # number of predictors
F = (SSR / p) / (SSE / (n - p - 1))
p_value = 1 - norm.cdf(F)
print('The SSE is', SSE)
print('The SSR is', SSR)
print("F-statistic:", F)
print("p-value:", p_value)

The SSE is 125.48533128079654
The SSR is 25.9557570091013
F-statistic: 6.618974630848736
p-value: 1.8084977959631487e-11


In [4]:
n_sims = 1000
n_rejections = 0
for i in range(n_sims):
    x = np.random.rand(n, 3)
    y = 1.0 + x[:,0] + 2*x[:,1] - 0*x[:,2] + norm.rvs(size=n)
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    p_value = model.pvalues[3]
    if p_value < 0.05:
        n_rejections += 1

print('The number of simulations with p-values < 0.05 is around %d, which is close to 0.05*1000.' % n_rejections)
print('The empirical type I error rate is %d/1000 = %.3f, which is close to the significance level of 0.05' % (n_rejections, n_rejections/n_sims))

The number of simulations with p-values < 0.05 is around 51, which is close to 0.05*1000.
The empirical type I error rate is 51/1000 = 0.051, which is close to the significance level of 0.05


In [5]:
# define x0
x0 = np.array([1, 0.3, 0.2, 0.7])
z = norm.ppf(0.975)
y_pred = model.predict(x0)
se_pred = np.sqrt(model.mse_resid + np.dot(x0, np.dot(model.cov_params(), x0.T)))
PI_low = y_pred - z * se_pred
PI_high = y_pred + z * se_pred
print('The predicted value for y0 is', y_pred[0])
print('The prediction interval for y0 is: [%f, %f]' % (PI_low,PI_high))

The predicted value for y0 is 1.8315038140904625
The prediction interval for y0 is: [-0.409945, 4.072953]


In [6]:
sample_sizes = [200, 500, 1000]
pred_int_widths = np.zeros(len(sample_sizes))
width = []
for i in range(len(sample_sizes)):
    n = sample_sizes[i]
    x = np.random.uniform(size=(n,3))
    eps = norm.rvs(size=n)
    y = 1.0 + x[:,0] + 2*x[:,1] - 0*x[:,2] + eps
    
    X = sm.add_constant(x)
    model = sm.OLS(y,X).fit()

    y_pred = model.predict(x0)
    se_pred = np.sqrt(model.mse_resid + np.dot(x0, np.dot(model.cov_params(), x0.T)))
    PI_low = y_pred - z * se_pred
    PI_high = y_pred + z * se_pred
    width.append(PI_high-PI_low)

print('The width of 95%% prediction intervals for y0 are: %f, %f and %f\n'% (width[0],width[1],width[2]) +
        'respectively for n = 200, 500 and 1000. The width does not approach to 0, but \n'+
        'gradually be close to its limit of 3.969969.' )

The width of 95% prediction intervals for y0 are: 3.967692, 3.947856 and 3.850112
respectively for n = 200, 500 and 1000. The width does not approach to 0, but 
gradually be close to its limit of 3.969969.


In [7]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

traffic = pd.read_csv('traffic.csv')
weather_types = ['Clouds', 'Clear', 'Snow']
data = traffic[traffic['weather_main'].isin(weather_types)]

model = ols('traffic_volume~weather_main',data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

SS_treatment = anova_table['sum_sq'][0]
df_treatment = anova_table['df'][0]
MS_treatment = SS_treatment/df_treatment
SS_error = anova_table['sum_sq'][1]
df_error = anova_table['df'][1]
MS_error = SS_error/df_error
SS_total = SS_treatment + SS_error
df_total = df_treatment + df_error
F = anova_table['F'][0]

print(anova_table)

                    sum_sq       df           F         PR(>F)
weather_main  2.548512e+09      2.0  338.384375  3.994626e-146
Residual      1.183486e+11  31428.0         NaN            NaN


In [8]:
data = {None: ['Treatment', 'Error', 'Total'],
        'SS': [SS_treatment, SS_error, SS_total],
        'df': [df_treatment, df_error, df_total],
        'MS': [MS_treatment, MS_error, None],
        'F': [F, None, None]}
df = pd.DataFrame(data)
df      #Fill in the ANOVA table

,None,SS,df,MS,F
0,Treatment,2.548512e+09,2.0,1.274256e+09,338.384375
1,Error,1.183486e+11,31428.0,3.765705e+06,NaN
2,Total,1.208971e+11,31430.0,NaN,NaN


In [9]:
import scipy.stats as stats
# Perform the hypothesis test
alpha = 0.05
f_critical = stats.f.ppf(q = 1-alpha, dfn = df_treatment, dfd = df_error)
print('(i) the degrees of freedom are (p-1, N-p). In this case, p-1 is (%d) and N-p is (%d)' % (df_treatment, df_error))
if F > f_critical:
    print(f'\n(ii) The calculated F-statistic ({F:.2f}) is greater than the critical value ({f_critical:.2f}).')
    print('Therefore, we reject the NULL hypothesis.')
    print('We conclude that the means under at least two of the three weather conditions are different.')
else:
    print(f"\n(ii) The calculated F-statistic ({F:.2f}) is not greater than the critical value ({f_critical:.2f}).")
    print('Therefore, we fail to reject the NULL hypothesis')
    print('We conclude that it is not enough to suggest the means are different.')

(i) the degrees of freedom are (p-1, N-p). In this case, p-1 is (2) and N-p is (31428)

(ii) The calculated F-statistic (338.38) is greater than the critical value (3.00).
Therefore, we reject the NULL hypothesis.
We conclude that the means under at least two of the three weather conditions are different.


: 